This code snippet will test the MP-DTC on the validation profile:

<img src="Figures/Validation_Profile.png" width="600">

The MP-DTC has been implemented according to
[MP-DTC Part 1](https://ieeexplore.ieee.org/abstract/document/6418026)
and
[MP-DTC Part 2](https://ieeexplore.ieee.org/abstract/document/6311466).

In [ ]:
import numpy as np
import sys, os
from pathlib import Path
sys.path.append(os.path.abspath(os.path.join('..')))
import gym_electric_motor as gem
from gym_electric_motor.reward_functions import WeightedSumOfErrors
from gym_electric_motor.physical_systems import ConstantSpeedLoad, ExternalSpeedLoad
from gym_electric_motor.reference_generators import WienerProcessReferenceGenerator, ConstReferenceGenerator, \
    MultipleReferenceGenerator, StepReferenceGenerator
from gym.core import Wrapper
from gym.spaces import Box, Tuple
import matplotlib.pyplot as plt
import matplotlib
from gym.wrappers import FlattenObservation
from tqdm import tqdm
from Plot_TimeDomain_torqueCtrl import calculate_performance_metrics
import h5py

In [ ]:
def test_profile_speed(t):
    """
    This function defines the speed profile of the validation episode.
    """
    lim = 12000 * 2 * np.pi / 60

    niveau0 = 00
    niveau1 = 0.15 * lim
    niveau2 = 0.5 * lim

    if t <= 0.05:
        omega = niveau0
    elif t <= 0.20:
        omega = (t - 0.05) * (niveau1 - niveau0) / 0.15 + niveau0
    elif t <= 1.3:
        omega = niveau1
    elif t <= 1.45:
        omega = (t - 1.3) * -2 * niveau1 / 0.15 + niveau1
    elif t <= 2.55:
        omega = - niveau1
    elif t <= 2.7:
        omega = (t - 2.55) * (niveau1 + niveau2) / 0.15 - niveau1
    elif t <= 3.8:
        omega = niveau2
    elif t <= 3.95:
        omega = (t - 3.8) * -2 * niveau2 / 0.15 + niveau2
    elif t <= 5.05:
        omega = - niveau2
    elif t <= 5.2:
        omega = (t - 5.05) * (niveau0 + niveau2) / 0.15 - niveau2
    else:
        omega = niveau0

    return omega

def test_profile_torque(step):
    """
    This function defines the torque profile of the validation episode.
    """

    if (step % 25000) <= 5000:
        torque_star = 0
    elif (step % 25000) <= 10000:
        torque_star = 3 / 8
    elif (step % 25000) <= 15000:
        torque_star = 3 / 4
    elif (step % 25000) <= 20000:
        torque_star = -3 / 4
    elif (step % 25000) <= 25000:
        torque_star = -3 / 8

    return torque_star

In [ ]:
class FcsMpcControllerPMSM:
    
    """
    Define the MP-DTC algorithm.
    MPC: Model Predictive Control, this algorithm uses a model of the plant system to determine optimal actions
    
    Notes: 
        MTPC: Maximum Torque Per Ampere
        MTPF: Maximum Torque Per Flux (does NOT consider motor resistance)
        MTPV: Maximum Torque Per Voltage (does consider motor resistance)
            if motor resistance can be neglected, MTPV is equivalent to MTPF (as presented in 
                https://ieeexplore.ieee.org/abstract/document/6418026 and
                https://ieeexplore.ieee.org/abstract/document/6311466)
    """

    def __init__(self, env):
        self.env = env

        params = env.env.physical_system.electrical_motor.motor_parameter
        self.tau = env.env.physical_system.tau
        self.subactions = -np.power(-1, env.env.physical_system._converter._subactions) * env.env.limits[7]

        # define the model parameters for the algorithm
        self.p = params['p']
        self.l_d = params['l_d']
        self.l_q = params['l_q']
        self.r_s = params['r_s']
        self.psi_p = params['psi_p']

        self.tau_d = self.l_d / self.r_s
        self.tau_q = self.l_q / self.r_s
        
        # define the constant input matrix of the system model
        self.B = np.array([[1 / self.l_d,            0],
                           [           0, 1 / self.l_q]])

    def control(self, observation):
        w_me = observation[0]
        w_el = w_me * self.p
        T = observation[1]
        
        i_a = observation[2]
        i_b = observation[3]
        i_c = observation[4]
        i_d = observation[5]
        i_q = observation[6]
        
        u_a = observation[7]
        u_b = observation[8]
        u_c = observation[9]
        u_d = observation[10]
        u_q = observation[11]

        cos_eps = observation[12]
        sin_eps = observation[13]
        T_ref = observation[14]

        eps = np.arctan2(observation[13], observation[12])

        # define the non-constant dynamic matrix A and disturbance matrix E
        A = np.array([[            - 1 / self.tau_d,  w_el * self.l_q / self.l_d],
                      [- w_el * self.l_d / self.l_q,            - 1 / self.tau_q]])

        E = np.array([[                             0],
                      [- w_el * self.psi_p / self.l_q]])

        i_dq_k = np.array([[i_d],
                          [i_q]])

        # predict the motor currents at the next sampling step, 
        # this needs to be done to incorporate the dead time of the plant in digitally controlled systems
        u_abc_k = [u_a, u_b, u_c]
        u_dq_k = np.transpose(np.array([env.env.physical_system.abc_to_dq_space(u_abc_k, epsilon_el=eps + w_el * self.tau * 0.5)]))
        # note that "@" is the matrix multiplication operator in python
        i_dq_k1 = i_dq_k + self.tau * (A @ i_dq_k + 
                                       self.B @ u_dq_k + 
                                       E)

        # loop over all applicable switching states to determine optimal action by trial
        # analytical optimization is (usually) not feasible in finite-control-set applications
        action_cost = []
        for i in range(8):
            # firstly, predict the currents that would result from the selected action
            u_abc_k1 = self.subactions[i]
            u_dq_k1 = np.transpose(np.array([env.env.physical_system.abc_to_dq_space(u_abc_k1, 
                                                                                     epsilon_el=eps + w_el * self.tau * 1.5)]))
            i_dq_k2 = i_dq_k1 + self.tau * (A @ i_dq_k1 + self.B @ u_dq_k1 + E)
            
            # then predict the corresponding torque and quadratic torque error
            T_k2 = 1.5 * self.p * (self.psi_p + (self.l_d - self.l_q) * i_dq_k2[0]) * i_dq_k2[1]
            # the higher the torque error, the higher the cost
            c_torque = (T_ref - T_k2) ** 2

            # punish violation of the current limit with a limitation cost term
            c_limit1 = 240 - np.sqrt(i_dq_k2[0] ** 2 + i_dq_k2[1] ** 2)
            if c_limit1 > 0:
                c_limit1 = 0
            else:
                c_limit1 = c_limit1 ** 2

            # punish positive d currents, only negative d currents are feasible in terms of efficiency
            c_limit2 = 2 * (self.l_d - self.l_q) / self.psi_p * i_dq_k2[0] + 1
            if c_limit2 > 0:
                c_limit2 = 0
            else:
                c_limit2 = c_limit2 ** 2

            u_dc = 350
            zeta = 1
            
            # punish operating points that are outside the controllable voltage ellipsis
            xi = np.sqrt((self.l_q * i_dq_k2[1]) ** 2 + (self.l_d * i_dq_k2[0] + self.psi_p) ** 2) \
                 - zeta * u_dc / np.sqrt(3) / np.abs(w_el)
            c_limit3 = 0
            if xi < 0:
                c_limit3 = xi ** 2

            # punish operating points on the left of the MTPF trajectory because efficiency would suffer
            theta = (psi_p ** 2 / self.l_q 
                     + self.psi_p * (2 * self.l_d / self.l_q - 1) * i_dq_k2[0] 
                     + self.l_d * (self.l_d / self.l_q - 1) * i_dq_k2[0] ** 2 
                     + self.l_q * (self.l_q / self.l_d - 1) * i_dq_k2[1] ** 2)
            c_limit4 = 0
            if theta < 0:
                c_limit4 = theta ** 2

            # check if operation point is on the left hand side of MTPC
            i_d_MPTC = (- self.psi_p / (2 * (self.l_d - self.l_q)) 
                       - np.sqrt((self.psi_p / (2 * (self.l_d - self.l_q))) ** 2 
                                 + i_dq_k2[1] ** 2))
            if i_d_MPTC > i_dq_k2[0]:
                left_MTPC = True
            else:
                left_MTPC = False

            # try to stick to the MTPC trajectory if not in field weakening operation
            c_attraction1 = (i_dq_k2[0] + (self.l_d - self.l_q) / self.psi_p * (i_dq_k2[0] ** 2 - i_dq_k2[1] ** 2)) ** 2
            
            # try to stick to the MTPF trajectory if in field weakening operation
            c_attraction2 = (np.sqrt((self.l_q / self.l_d * i_dq_k2[1]) ** 2 + (i_dq_k2[0] + self.psi_p / self.l_d) ** 2)
                          - zeta * u_dc / np.sqrt(3) / np.abs(w_el) / self.l_d) ** 2
            
            # decide whether field weakening applies
            if c_attraction2 < c_attraction1 and left_MTPC:
                c_attraction = c_attraction2
            else:
                c_attraction = c_attraction1

            # these weighting / boundary parameters have been optimized for the given validation profile and motor
            # for different motor parameterization a different weighting might be better
            lambda_attraction = 0.325
            lambda_limit = 2000
            omega_boundary = 6
            
            # decide whether voltage limits have to be considered or not
            if np.abs(w_me) >= omega_boundary:
                cost = c_torque + lambda_attraction * c_attraction + lambda_limit * (c_limit1 + c_limit2 + c_limit3 + c_limit4)
            else:
                cost = c_torque + lambda_attraction * c_attraction1 + lambda_limit * (c_limit1 + c_limit2 + c_limit4)

            # the predicte costs for all switching states are written into an array
            action_cost.append(cost)

        # use the action that is cheapest in terms of cost
        action = np.argmin(action_cost)
        return action


In [ ]:
class TranformObservationWrapper(Wrapper):
    """
    The following environment considers the dead time in the real-world motor control systems.
    The real-world system changes its state, while the agent calculates the next action based on a previoulsly measured
    observation. Therefore, for the agents it seems as if the applied action effects the state one step delayed.
    (with a dead time of one time-step)

    For complete observability of the system at each time-step we append the last played action of the agent to the
    observation, because this action will be the one that is active in the next step.
    """
    def __init__(self, environment):
        super().__init__(environment)
        self.prev_voltage_state = np.zeros(5)
        self.observation_space = Tuple((Box(
            np.concatenate((environment.observation_space[0].low[ 0:12], [-1, -1], [-1])),
            np.concatenate((environment.observation_space[0].high[0:12], [ 1,  1], [ 1]))
        ), environment.observation_space[1]))

        self.subactions = -np.power(-1, self.env.physical_system._converter._subactions)


    def step(self, action):

        (state, ref), rew, term, info = self.env.step(action)

        self._obs_logger = np.concatenate((state, ref))

        eps = state[12] * np.pi
        angle_scale = 1
        angles = [angle_scale * np.cos(eps), angle_scale * np.sin(eps)]
        torque_error = [(ref[0] - state[1]) / 2]

        u_abc = self.subactions[action]
        u_dq = self.env.physical_system.abc_to_dq_space(u_abc, epsilon_el=eps)
        self.now_requested_voltage = np.concatenate((u_abc, u_dq))

        state = np.concatenate((state[0:7], 
                                self.now_requested_voltage, 
                                angles))

        return (state, ref), rew, term, info

    def reset(self, **kwargs):
        state, ref = self.env.reset()

        self._obs_logger = np.concatenate((state, ref))

        eps = state[12] * np.pi
        angle_scale = 1
        angles = [angle_scale * np.cos(eps), angle_scale * np.sin(eps)]
        torque_error = [(ref[0] - state[1]) / 2]

        u_abc = self.subactions[0]
        u_dq = self.env.physical_system.abc_to_dq_space(u_abc, epsilon_el=eps)
        self.now_requested_voltage = np.concatenate((u_abc, u_dq))

        state = np.concatenate((state[0:7], 
                                self.now_requested_voltage, 
                                angles))

        return (state, ref)

torque_ref_generator = ConstReferenceGenerator(reference_state='torque',
                                               reference_value=0)

p = 3  # [p] = 1, nb of pole pairs
r_s = 17.932e-3  # [r_s] = Ohm, stator resistance
l_d = 0.37e-3  # [l_d] = H, d-axis inductance
l_q = 1.2e-3  # [l_q] = H, q-axis inductance
psi_p = 65.65e-3  # [psi_p] = Vs, magnetic flux of the permanent magnet

motor_parameter = dict(p=p,  # [p] = 1, nb of pole pairs
                       r_s=r_s,  # [r_s] = Ohm, stator resistance
                       l_d=l_d,  # [l_d] = H, d-axis inductance
                       l_q=l_q,  # [l_q] = H, q-axis inductance
                       psi_p=psi_p,  # [psi_p] = Vs, magnetic flux of the permanent magnet
                       )  # BRUSA

u_sup = 350
nominal_values = dict(omega=4000*2*np.pi/60,
                      i=230,
                      u=u_sup)

limit_values = nominal_values.copy()
limit_values["i"] = 270
limit_values["torque"] = 200


env = gem.make("PMSMDisc-v1",
               motor_parameter=motor_parameter,
               nominal_values=nominal_values,
               limit_values=limit_values,
               u_sup=u_sup,
               load=ExternalSpeedLoad(speed_profile=test_profile_speed, tau=50e-6),
               tau=50e-6,
               reward_function=WeightedSumOfErrors(observed_states=None,
                                                          reward_weights={'torque': 1},
                                                          gamma=0),
               reference_generator=torque_ref_generator,
               ode_solver='scipy.solve_ivp',
               dead_time=True
               )

env = FlattenObservation(TranformObservationWrapper(env))

params = env.env.physical_system.electrical_motor.motor_parameter
limits = np.concatenate((env.env.env.limits[0:12],
                         [1, 1],
                         [env.env.env.limits[1]]))
tau = env.env.physical_system.tau

controller = FcsMpcControllerPMSM(env=env)

observation = env.reset()
OBSERVATION = np.transpose(np.array([observation]))
ACTION = np.array([0])
TIME = np.array([0])

TORQUE_ERROR = np.array([0])

# simulate the closed loop system (controller and plant)
step_horizon = 130000 
for idx in tqdm(range(step_horizon)):
    time = idx * tau

    action = controller.control(observation * limits)
    observation, reward, done, _ = env.step(action)

    # write the test profile reference into the corresponding observation index (overwriting the builting GEM reference generator)
    observation[14] = test_profile_torque(idx)

    # log the state observations and actions in an array to plot them later
    OBSERVATION = np.append(OBSERVATION, np.transpose([observation]), axis=1)
    ACTION = np.append(ACTION, action)
    TIME = np.append(TIME, time)

# calculate the performance metrics
mean_reward = calculate_performance_metrics(OBSERVATION[5], 
                                            OBSERVATION[6], 
                                            OBSERVATION[1], 
                                            OBSERVATION[14])

# save the measurement data to folder "MP_DTC"
folder_name = "MP_DTC"
Path(folder_name).mkdir(parents=True, exist_ok=True)
with h5py.File(folder_name + "/" + "MP_DTC_validation_episode_0" + ".hdf5", "w") as f:
    print(tau)
    tau = f.create_dataset("tau", data=tau)
    lim = f.create_dataset("limits", data=limits)

    obs = f.create_dataset("observations", data=np.transpose(OBSERVATION))
    rews = f.create_dataset("rewards", data=np.zeros_like(TIME))
    acts = f.create_dataset("actions", data=ACTION)

In [ ]:
from Plot_TimeDomain_torqueCtrl import plot_episode

# plot the 
plot_episode(training_folder = "MP_DTC",
             episode_number = 0,
             episode_type = "MP_DTC_validation_episode")